<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Using a Local LLM</h1>

Below is an example of using a local LLM to perform evals. In this example we will be using [ollama](https://ollama.com/)

In [ ]:
!pip install -qq "arize-phoenix-evals>=0.0.5" "litellm"

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import os

from phoenix.evals import LiteLLMModel

os.environ["OLLAMA_API_BASE"] = "http://localhost:11434"

model = LiteLLMModel(model="ollama/llama3")

In [ ]:
model("Hello, world!")

In [ ]:
from phoenix.evals import download_benchmark_dataset

df = download_benchmark_dataset(
    task="binary-relevance-classification", dataset_name="wiki_qa-train"
)
df.head()

In [ ]:
from phoenix.evals import (
    RAG_RELEVANCY_PROMPT_RAILS_MAP,
    RAG_RELEVANCY_PROMPT_TEMPLATE,
    llm_classify,
)

N_EVAL_SAMPLE_SIZE = 100

df_sample = df.sample(n=N_EVAL_SAMPLE_SIZE)

df_sample = df_sample.rename(
    columns={
        "query_text": "input",
        "document_text": "reference",
    },
)

rails = list(RAG_RELEVANCY_PROMPT_RAILS_MAP.values())

relevance_df = llm_classify(
    dataframe=df_sample,
    template=RAG_RELEVANCY_PROMPT_TEMPLATE,
    model=model,
    rails=rails,
    concurrency=20,
)

In [ ]:
relevance_df.head(20)